In [1]:
cd ..

/home/harsh/projects/sembly-round2-harsh


In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from tqdm import tqdm

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

/home/harsh/miniconda3/envs/pyt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df=pd.read_csv("task-1/data/cleaned_data.csv")
y_true=df['is_duplicate'][:200]

In [5]:
question1=df["question1"][:200].tolist()
question2=df["question2"][:200].tolist()

In [6]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

In [7]:
model

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0): MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_features

In [8]:
all_questions=question1+question2

In [9]:
tokens = {'input_ids': [], 'attention_mask': []}  

In [10]:
for sentence in tqdm(all_questions):
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=200,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])


100%|██████████████████████████████████████████████████████| 400/400 [00:00<00:00, 5632.49it/s]


In [11]:
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [ ]:
outputs = model(**tokens)
outputs.keys()

In [ ]:
embeddings = outputs.last_hidden_state
embeddings


In [ ]:
attention_mask = tokens['attention_mask']
attention_mask.shape

In [ ]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

In [ ]:
masked_embeddings = embeddings * mask
masked_embeddings.shape

In [ ]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

In [ ]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

In [ ]:
mean_pooled = summed / summed_mask

In [ ]:
mean_pooled.shape

In [ ]:
question1_vectors=mean_pooled[:int(len(mean_pooled)/2)]
question2_vectors=mean_pooled[int(len(mean_pooled)/2):]